<a href="https://colab.research.google.com/github/tpmccrea/GradStudentShowcase/blob/master/GradStudentShowcase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import ee

In [0]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/xAHb7ys6rsNaE9XuyXd7wzwNUBaKOVPjMtAd29LlZZQa8FwJH7j1T4U

Successfully saved authorization token.


In [24]:
#Import libraries
import ee
import folium

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = "Map Data © Google Earth Engine",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Fetch an elevation model.
OpticalDepth = ee.Image('MODIS/006/MOD08_M3/A2018091')
GlobalTemp = ee.Image('MODIS/006/MOD11A1/2018_09_01')

# Set visualization parameters.
vis_params = {
  'min': -50,
  'max': 800,
  'palette': ['0500ff', ' 00d4ff', ' 00ff10', ' FFd200', 'FF1e00']}

vis_params1 = {
  'min': 12000,
  'max': 16000,
  'palette': ['0500ff', ' 00d4ff', ' 00ff10', ' FFd200', 'FF1e00']}

# Create a folium map object.
my_map = folium.Map(location=[20, 0], zoom_start=3, height=500)

# Add the elevation model to the map object.
my_map.add_ee_layer(OpticalDepth.select('Aerosol_Optical_Depth_Land_Ocean_Mean_Mean'), vis_params, 'Mean Aerosol Optical Depth')
my_map.add_ee_layer(GlobalTemp.select('LST_Day_1km'), vis_params1, 'Global Surface Temp')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

# Export Map 
my_map.save('index1.html')

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/xAFTSa0mHNaj87SdIGzL8dJgWt6ETgguS8qMif9lAv8ckviY-Hwpru0

Successfully saved authorization token.
